# Problemas de estado mutable compartido
En esta sección presentaremos dos problemas, en Java, cada problema contiene el problema del **estado mutable compartido**. La idea es solucionar cada estado mutable compartido utilizando la técnica de copia vista en el vídeo anterior.

## Problema 1. (Sistema de transacciones)

Un sistema de transacciones registra las operaciones en moneda extranjera, supogamos que tenemos un clase llamada `ForeingEx`chages que representa divisas en moneda extranjera y su valor multiplicativa de conversión actual. 

In [10]:
public class ForeignEx {
   private double value;
   public ForeignEx(double value) {
      this.value = value;
   }
   public double getValue() {
      return value;
   }
   public void setValue(double value) {
      this.value = value;
   }
   public String toString() {
      return "" + value;
   }
}

Tenemos un sistema de transacciones a través de la clase `Transaction` que registra la cantidad de divisa extranjera en conjunto con el valor de la divisa extranjera. De la siguiente forma:

In [15]:
public class Transaction {
    private double amount;
    private ForeignEx forEx;
    public Transaction(double amount, ForeignEx forEx) {
       this.amount = amount;
       this.forEx = forEx;
    }
    public double getTransValue() {
       return amount * forEx.getValue();
    }
    public String toString() {
       return "amount: " + " Foreign Exchange: " + fe + "\nTotal: " + getTransValue() + "\n";
    }
}

La siguiente es una operación de registrar varias operaciones en monto de moneda con dicha divisa y procesarla.

In [19]:
double amounts[] = { 1.2, 3.4, 5.6 };
ForeignEx fe = new ForeignEx(10.1);
List<Transaction> transDay1 = new ArrayList<>();
for (int i = 0; i < amounts.length; i++) {
    transDay1.add(new Transaction(amounts[i],fe));
}
System.out.println(transDay1);

[amount:  Foreign Exchange: 10.1
Total: 12.12
, amount:  Foreign Exchange: 10.1
Total: 34.339999999999996
, amount:  Foreign Exchange: 10.1
Total: 56.559999999999995
]


Suponga que se registra otro día de transacciones, pero se cambia el valor de la divisa:

In [20]:
double amountsDay2[] = { 2.1, 4.9, 3.2 };
fe = new ForeignEx(20.1);
List<Transaction> transDay2 = new ArrayList<>();
for (int i = 0; i < amountsDay2.length; i++) {
    transDay2.add(new Transaction(amounts[i],fe));
}
System.out.println(transDay2);

[amount:  Foreign Exchange: 20.1
Total: 24.12
, amount:  Foreign Exchange: 20.1
Total: 68.34
, amount:  Foreign Exchange: 20.1
Total: 112.56
]


Aparentemente todo se encuentra bien. Si imprimimos las transacciones del día uno. Algo pasa.

In [21]:
System.out.println(transDay1);

[amount:  Foreign Exchange: 20.1
Total: 12.12
, amount:  Foreign Exchange: 20.1
Total: 34.339999999999996
, amount:  Foreign Exchange: 20.1
Total: 56.559999999999995
]


## Problema 2. (Sistema de registro de versiones)

Vamos a presentar un problema que es llevar una lista de las diferentes versiones que tenemos almacenadas en un posible fichero.
Empecemos por las clase `Version` la idea de esta clase es tener un registro del número de versión actual (por facilidad con los números de versiones que consta de al menos de tres números en secuencias diferentes X.Y.Z, la vamos a simplicar con único número). Como se puede observar nuestra clase `Version` es una clase muy simple que simplemente tiene dos constructores: Uno por omisión que inicia la secuencia en 0 y otro que inicia con un número de versión que iniciar (Se espera que el número de versión sea un número mayor e igual a 0).

public class Version {
  private int versionNumber;
  public Version(int versionNumber) {
     assert versionNumber >= 0;
     this.versionNumber = versionNumber;
  }
  public Version() {
     this(0);
  }
  public int getVersionNumber() {
     return versionNumber;
  }
  public int incrVersionNumber() {
      return versionNumber++;
  }
  public String toString() {
      return "" + versionNumber;
  }
}

Podemos probar en el siguiente código que va incrementando el número de versión varias veces e imprimiéndolo.

In [13]:
Version v = new Version();
System.out.println(v);
v.incrVersionNumber();
System.out.println(v);
v.incrVersionNumber();
System.out.println(v);

0
1
2


Ahora con este código vamos a implementar una bitácora de versiones sobre un fichero. Normalmente la bitácora de versiones lleva un registro de cada versión en la que ha habido cambios en un proyecto. Para ellos vamos a utilizar una lista (`List`) de versiones donde se guarda cada una de las versiones por las que ha pasado nuestro fichero. Nuestro proyecto consta de un solo fichero. El siguiente código genera la bitácora:

In [17]:
List<Version> log = new ArrayList<>();
Version fileVersion = new Version();

Vamos a simular el comportamiento de la bitácora, realizando una secuencia de 5 cambios a través del siguiente procedimiento que simula la ejecución de n versiones.

In [20]:
static void seqChanges(int n) {
    for (int i = 0; i < n; i++) {
        System.out.println("Current version: " + fileVersion.getVersionNumber());
        log.add(fileVersion);
        fileVersion.incrVersionNumber();
        System.out.println("New version: " + fileVersion.getVersionNumber());
    }
}

Probemos que todo (aparentemente) funciona bien:

In [21]:
seqChanges(5);

Current version: 5
New version: 6
Current version: 6
New version: 7
Current version: 7
New version: 8
Current version: 8
New version: 9
Current version: 9
New version: 10


Todo bien, hemos hecho los cambios correspondientes y se han correctamente la versiones `[0,1,2,3,4]`. El siguiente valor a registrar es el 5. Entonces, miremos el siguiente código que debe imprimir lo mismo.

In [22]:
System.out.println(log);

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


In [ ]:
public class Version {
  private int versionNumber;
  public Version(int versionNumber) {
     assert versionNumber >= 0;
     this.versionNumber = versionNumber;
  }
  public Version() {
     this(0);
  }
  public int getVersionNumber() {
     return versionNumber;
  }
  public Version incrVersionNumber() {
      return new Version();
  }
  public String toString() {
      return "" + versionNumber;
  }
}

Ahora el procedimiento seqChanges cambia, en esta caso la función recibe tres valores el número `n` de veces y el valor de la versión actual. Y la función retorna un Pair donde la clave es el valor siguiente de la versión, y el valor es de la bitácora modificada. El siguiente es parte del cuerpo de la función.

In [24]:
class Pair<K,V> {
   private K key;
   private V value;
   Pair(K key, V value) {
      this.key = key;
      this.value = value;
   }
   K getKey() {
      return key;
   }
   V getValue() {
      return value;
   }
}

static Pair<Version,List<Version>> seqChange(int n, Version actual) {
    return new Pair(actual, new ArrayList<>());
}

La función `seqChange` es una versión muy similar a la anterior, no es una **función pura**.

